# 1. [60+10] Language and Topic models

A common suggestion to users for coming up with good queries is to think of words that would likely appear in a relevant document, and to use those words as the query. The language modeling approach to IR directly models this idea: a document is a good match to a query if the document model is likely to generate the query, which will in turn happen if the document contains the query words often. 

You will score documents with respect to user query using language models and also get some experience with topic modelling.

## 1.0. [5] Loading data

We use the dataset we already used once - [this topic-modeling dataset](https://code.google.com/archive/p/topic-modeling-tool/downloads) ([or from github](https://github.com/IUCVLab/information-retrieval/blob/main/datasets/topic-modelling.zip)).

In [1]:
import os
# Idea taken from: https://www.guru99.com/tokenize-words-sentences-nltk.html
from nltk import word_tokenize

path = 'topic-modelling/'
# Taken from: https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
dirpath, dirnames, filenames = next(os.walk(path))

all_data = []
categories =  {
    "testdata_braininjury_10000docs.txt" : "brain_injury",
    "testdata_news_economy_2073docs.txt" : "economy",
    "testdata_news_fuel_845docs.txt"     : "fuel",
    "testdata_news_music_2084docs.txt"   : "music"
}

for file in filenames:
    with open(path + file, 'r') as f:
        sentences = f.read().split('\n')
        all_data.extend([(sent, str(categories[file]) + " " + str(doc_id)) for doc_id, sent in enumerate(sentences) if sent])

print("# of documents", len(all_data))
assert len(all_data) == 15002

# of documents 15002


## 1.1. [55] Ranking Using Language Models
Our goal is to rank documents by $P(d|q)$, where the probability of a document is interpreted as the likelihood that it is relevant to the query. 

Using Bayes rule: $P(d|q) = \frac{P(q|d)P(d)}{P(q)}$

$P(q)$ is the same for all documents, and so can be ignored. The prior probability of a document $P(d)$ is often treated as uniform across all $d$'s and so it can also be ignored. What does it mean? 

It means that computing $P(q|d)$ for different documents we can compare how relevant are they to the query. How can we estimate $P(q|d)$?

$P(q|d)$ can be estimated as:

![](https://i.imgur.com/BEIMAC1.png)

where $M_d$ is the language model of document $d$, $tf_{t,d}$ is the term frequency of term $t$ in document $d$, and $L_d$ is the number of tokens in document $d$. That is, we just count up how often each word occurred, and divide by the total number of words in the document $d$.

### 1.1.1. [15] Build TDM (or DTM)

The first thing we need to do is to build a term-document matrix for tour dataset.

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()

dataset = [x[0] for x in all_data]
DTM = vect.fit_transform(dataset).toarray()
print("Vocabulary:", vect.get_feature_names())

Vocabulary: ['000l', '000m', '000th', '03rd', '0as', '0dpt', '0gy', '0mm', '0yrs', '1000m', '1000w', '100ab', '100b', '100bb', '100beta', '100g', '100percent', '100th', '101st', '1027g', '103rd', '1040a', '1040s', '104th', '105th', '106s', '106th', '107th', '108th', '109th', '10am', '10fr', '10g', '10million', '10mm', '10mo', '10mwt', '10mwts', '10p10', '10s', '10th', '10x', '10y', '10years', '110million', '110mm', '110th', '112th', '113th', '114th', '115th', '116th', '117th', '118b', '118th', '1199s', '11a', '11b', '11c', '11million', '11th', '120min', '120th', '1223willsmith', '123i', '1240ut', '125cc', '125d', '125i', '125ml', '125ms', '125s', '125th', '126th', '127th', '128th', '1296c', '129th', '12h', '12m', '12million', '12mo', '12p2', '12p6', '12p70', '12th', '12u', '12v2', '1300s', '130f', '130j', '130million', '130r', '130s', '131xe', '135th', '136million', '137th', '138th', '139th', '13c', '13c2', '13hz', '13percent', '13th', '1400s', '1400smith', '144a', '145th', '148million

In [3]:
# Get all documents names as in DTM
doc_id = [x[1] for x in all_data]
# Get all features
features = vect.get_feature_names()

### 1.1.2. [30] Smoothing

Now, you need to implement the abovementioned logic in the `lm_rank_documents` function below. Do you see any potential problems?

Yes, data sparsity - we don't expect to meet each term in each doc, so, in most cases, we will get zero scores, which is not what we really want.

The solution is smooting.

One option is *[additive smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)* - adding a small number (0 to 1)D to the observed counts and renormalizing to give a probability distribution.

Another option is called [Jelinek-Mercer smoothing](http://mlwiki.org/index.php/Smoothing_for_Language_Models#Jelinek-Mercer_Smoothing) - a simple idea that works well in practice is to use a mixture between a document-specific distribution and distribution estimated from the entire collection:

![](https://i.imgur.com/8Qv41Wp.png)

where 0 < λ < 1 and $M_c$ is a language model built from the entire document collection.

Refer to [*Chapter 12*](https://nlp.stanford.edu/IR-book/html/htmledition/language-models-for-information-retrieval-1.html) for the detailed explanation.


You are going to apply both in your `lm_rank_documents` function. This function takes TDM or DTM as an input, and ranks all documents "building" a language model for each document, returning relative probabilities of query being generated by a document as a document's score.

In [4]:
import numpy as np
from nltk import word_tokenize

def lm_rank_documents(query, smoothing='additive', param=0.001):
    # TODO: score each document in tdm using this document's language model
    # implement two types of smoothing. Looks up term frequencies in tdm
    # return document scores in a convenient form
    # param is alpha for additive / lambda for jelinek-mercer
    scores = []
    terms_id = [features.index(term) for term in word_tokenize(query.lower()) if term in features]

    if smoothing == 'additive':
        DTM_added = (DTM+param)
        DTM_added /= np.sum(DTM_added, axis=1).reshape(-1,1)

        for doc in range(len(doc_id)):
            new_score = 1
            for idx in terms_id:
                new_score = new_score * DTM_added[doc][idx]
            scores.append((doc_id[doc], new_score))
    else:
        columns_sum = DTM.sum(axis=0)
        for doc in range(len(doc_id)):
            new_score = 1
            term_score = 1
            for idx in terms_id:
                new_score = new_score * DTM[doc][idx]
                term_score = term_score*columns_sum[idx]
            score = param*new_score + (1-param)*term_score
            scores.append((doc_id[doc], score))

    scores.sort(key = lambda tup: tup[1], reverse=True)
    return scores[:5]

### 1.1.3. [10] Testing

Check if this type of ranking gives meaningful results. For each query output document `category`, `doc_id`, `score`, and the *beginning* of the document, as it is shown below. Analyze if categories and contents match the queries. 

In [5]:
def process_query(raw_query):
    # TODO: process user query and print search results including document category, id, score, and some part of it
    results = lm_rank_documents(raw_query)
    print('user query: ' + raw_query)
    print('\n')
    print('search results:')
    for result in results:
        doc = result[0]
        print(str(doc) + ' ' + '%.8f' %result[1])
        print(all_data[doc_id.index(doc)][0][:150])
    pass
    

user_queries = ["piano concert", "symptoms of head trauma", "wall street journal"]
for q in user_queries:
    process_query(q)
    print("\n")

user query: piano concert


search results:
music 916 0.00003733
sometimes the most satisfying renovation the one that doesn happen two years ago geoffrey menin bought loft fifth avenue near 20th street drawn the la
music 890 0.00003093
playing carnegie hall concert performer dream when word leaked out that america most famous concert hall was looking for new talent husband and wife c
music 670 0.00002784
igor kipnis the virtuoso harpsichordist whose busy concert recording career made him the instrument most ardent cheerleader died wednesday his connect
music 1994 0.00000953
leo ornstein russian born composer and pianist who the early 20th century was leading figure the american avant garde died feb green bay wis was eithe
music 8 0.00000863
felt like was going church marry guy never met said the jazz violinist regina carter the metaphorical church was the carlo felice opera house here the


user query: symptoms of head trauma


search results:
brain_injury 1490 0.00001153
conversion s

Sample results can look like this (if collapsed, click on 3 dots):

```
user query: piano concert

search results:
music 13330 0.012384164490679759
atlanta prominent midtown intersection one step closer becoming major cultural landmark the atlanta ...
economy 11335 0.012384164490679759
atlanta prominent midtown intersection one step closer becoming major cultural landmark the atlanta ...
music 12926 0.011382499792705511
felt like was going church marry guy never met said the jazz violinist regina carter the metaphorica...
music 14390 0.010661589922122
hailed los angeles brightest flower its flashiest ship sail its keenest architectural triumph perhap...
music 13818 0.010549141787975117
everything was finished sept the super bowl logo would reflection new orleans featuring streetcar an...


user query: symptoms of head trauma

search results:
brain_injury 7319 0.06022877378376099
the direct economic burden blunt and penetrating trauma managed care population background although ...
brain_injury 6987 0.05854539395767944
history reported head trauma sample women substance abuse treatment objectives determine the prevale...
brain_injury 5257 0.05760140208255336
violent head trauma china report cases background the occurrence violent trauma has recently increas...
brain_injury 1536 0.055365767080148634
mild head trauma and chronic headaches returning soldiers objective determine the incidence and type...
brain_injury 8874 0.05379997937839304
maxillofacial trauma major trauma patients background trauma has been identified major public health...


user query: wall street journal

search results:
economy 11294 0.027288833622119528
these business stories for release tuesday january are moving today clients the new york times news ...
economy 11295 0.027288833622119528
these business stories for release tuesday january are moving today clients the new york times news ...
music 14641 0.026716049665405375
these feature stories are moving today clients the new york times news service stories are for relea...
music 14640 0.026716049665405375
these feature stories are moving today clients the new york times news service stories are for relea...
economy 11297 0.025763725974814314
these feature stories are moving today clients the new york times news service stories are for relea...
```

## 1.2. [+10] Topic modeling

Now let's use *Latent Dirichlet Allocation* to identify topics in this collection and check if they match the original topics (fuel, economy, etc.). Go through the tutorial [here](https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0) and apply the ideas there to our dataset. 

In [6]:
# TODO: apply LDA to our dataset and output the resulting categories 


We expect to see something like this (if collapsed, click on 3 dots):

```
Topic #0:
brain injury patients tbi traumatic study cerebral results severe group cognitive clinical pressure imaging following outcome control using children test

Topic #1:
new said york news atlanta like times year service time people undated just music journal constitution city says com years

Topic #2:
patients injury injuries trauma head study results traumatic brain treatment cases patient fractures years case outcome methods clinical tbi surgery

Topic #3:
said year bush percent new enron company president government people economy years million state companies states economic united time billion
```

# 2. [40+10] Sugges_

One of the strategies to improve user experience is to provide user with hints, or, otherwise, to autocomplete his queries. Let's consider suggest.

Today we will practice generating suggestions using [Trie](https://en.wikipedia.org/wiki/Trie) datastructure (prefix tree), see the example below.

Plan of your homework:

1. Build Trie based on real search query data, provided by AOL company;
2. Generate suggestion based on trie;
3. Measure suggestion speed;
4. (+) Optionally add spellcheck to suggest.


![image](https://www.ritambhara.in/wp-content/uploads/2017/05/Screen-Shot-2017-05-01-at-4.01.38-PM.png)

## 2.0. Install Trie data structure support

You are free to use any library implementation of Trie, as well as the one we suggest (read the docs before asking any questions!): https://github.com/google/pygtrie

In [7]:
!pip install pygtrie

### 2.0.1. Check it works and understand the example

In [8]:
import pygtrie
t = pygtrie.CharTrie()

# trie can be considered as a form of organizing a set of map
t["this is 1"] = "A"
t["this is 2"] = "B"
t["that is 3"] = "C"

print(t)

# "this" string is present in a set
n = t.has_node('this') == pygtrie.Trie.HAS_VALUE
# "this" prefix is present in a set
s = t.has_node('this') == pygtrie.Trie.HAS_SUBTRIE

print(f"Node = {n}\nSubtree = {s}")

# iterate a subtree
for key, val in t.iteritems("this"):
    print(key, '~', val)

CharTrie(this is 1: A, this is 2: B, that is 3: C)
Node = False
Subtree = True
this is 1 ~ A
this is 2 ~ B


## 2.1. Build a trie upon a dataset

### 2.1.1. [5] Read dataset

Download the [dataset](https://github.com/IUCVLab/information-retrieval/tree/main/datasets/aol) (we provide only the first part of the original data for simplicity (~3.5 mln queries)).

Explore the data, see readme file. Load the dataset.

In [9]:
import pandas as pd

aol_data =pd.read_csv("user-ct-test-collection-01.txt", encoding='utf-8-sig', sep="\t")

#TODO: Read the dataset, e.g. as pandas dataframe

assert aol_data.shape[0] == 3558411, "Dataset size doesnt' match"

### 2.1.2. [10] Build Trie

We want suggest function to be **non-sensitive to stop words** because we don't want to upset the user if he confuses/omits prepositions, for example. Consider `public events in Innopolis` vs `public events at Innopolis` or `public events Innopolis` - they all mean the same.

Build Trie based on the dataset, **storing query statistics such as query frequency, urls and ranks in nodes**. Some queries may not have associated urls, others may have multiple ranked urls, consider this.

In [10]:
aol_array = aol_data.to_numpy()

In [11]:
from collections import Counter
import pygtrie

# Used this: https://towardsdatascience.com/python-pro-tip-start-using-python-defaultdict-and-counter-in-place-of-dictionary-d1922513f747
# Used this: https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas
# Used: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop = stopwords.words('english')

def query_prep(raw_query):
    res = ' '.join([term for term in raw_query.split() if term not in stop])


aol_trie = pygtrie.CharTrie()

query_count = Counter()
statistics = {}

for row in aol_array:
    query = str(row[1])
    query_count.update([query])
    if query in statistics:
        statistics[query].append((row[4], row[3]))
    else:
        statistics[query]= [(row[4], row[3])]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ravio_o/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
import math
length = len(aol_array)

for q, qc in query_count.items():
    urls = [x for x in statistics[q] if not math.isnan(x[1])]
    aol_trie[q] = (qc/length, urls)

In [13]:
# test trie
bag = []
for key, val in aol_trie.iteritems("sample q"):
    print(key, '~', val)
    
    #NB: here we assume you store urls in a list field. But you can do something different. 
    bag += [url for url, rank in val[1]]
    
    assert "sample question" in key, "All examples have `sample question` substring"
    assert key[:len("sample question")] == "sample question", "All examples have `sample question` starting string"

for url in ["http://www.surveyconnect.com", "http://www.custominsight.com", 
            "http://jobsearchtech.about.com", "http://www.troy.k12.ny.us",
            "http://www.flinders.edu.au", "http://uscis.gov"]:
    assert url in bag, "This url should be in a try"

sample question surveys ~ (1.4051215556606586e-06, [('http://www.surveyconnect.com', 7.0), ('http://www.custominsight.com', 4.0), ('http://www.askemployees.com', 10.0), ('http://www.lg-employers.gov.uk', 1.0)])
sample questions for immigration interview ~ (2.810243111321317e-07, [])
sample questions for interview ~ (2.810243111321317e-07, [('http://www.quintcareers.com', 1.0)])
sample questions for family interview ~ (8.430729333963952e-07, [('http://www.grandparents-day.com', 2.0), ('http://www.quintcareers.com', 5.0), ('http://jobsearchtech.about.com', 3.0)])
sample questions for us citizenship test ~ (2.810243111321317e-07, [('http://uscis.gov', 1.0)])
sample questions sociology race and ethnicity ~ (2.810243111321317e-07, [])
sample questions biology ~ (5.620486222642634e-07, [('http://www.utexas.edu', 3.0), ('http://www.troy.k12.ny.us', 6.0)])
sample questionarie teaching evaluation ~ (2.810243111321317e-07, [])
sample questionnaire teaching evaluation ~ (1.4051215556606586e-06, [

## 2.2. [10] Write a suggest function which is non-sensitive to stop words

Suggest options for user query based on Trie you just built.
Output results sorted by frequency, print query count for each suggestion. If there is an url available, print the url too. If multiple url-s are available, print the one with the highest rank (the less the better).

In [14]:
def complete_user_query(query, trie, top_k=5):
    query_prep = ' '.join([term for term in query.split() if term not in stop])
    try:
        results = [(key,value) for key, value in trie.iteritems(query_prep)]
    except:
        print('No suggestions for query: ' + query)
        return []

    results.sort(key = lambda x: x[1][0], reverse = True)
    for result in results[:top_k]:
        key = result[0]
        print('Suggestion: \'' + str(key) + '\' has query count: ' + str(query_count[key]))
        urls = sorted(result[1][1], key = lambda x: x[1])
        if urls:
            print('URL ' + str(urls[0][0]))

    return  [key for key,_ in results[:top_k]]

        
inp = "trie"
print("Query:", inp)
print("Results:")
res = complete_user_query(inp, aol_trie)

#NB we assume you return suggested query string only
assert res[0] == "tried and true tattoo"
assert res[1] == "triest" or res[1] == "triethanalomine"

Query: trie
Results:
Suggestion: 'tried and true tattoo' has query count: 5
URL http://www.triedntruetattoo.com
Suggestion: 'triest' has query count: 3
Suggestion: 'triethanalomine' has query count: 3
URL http://avalon.unomaha.edu
Suggestion: 'tried and failed' has query count: 2
Suggestion: 'tried and truechildren's consignment sale' has query count: 1


## 2.3. [10] Measure suggest speed ##

Check how fast your search is working. Consider changing your code if it takes too long on average.

In [15]:
from time import process_time

inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell", 
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]

all = process_time()
for term in inp_queries:
    start_query = process_time()

    complete_user_query(term, aol_trie)
    print('For query: \'' + term + '\' process time is: ' + str(process_time()-start_query))
    print('\n')


print('For all queries process time is: ' + str((process_time() - all) / len(inp_queries)))
#TODO: measure avg execution time (in milliseconds) per query and print it out

Suggestion: 'information clearing house' has query count: 94
URL http://www.informationclearinghouse.info
Suggestion: 'information on training puppy' has query count: 72
URL http://www.101-dog-training-tips.com
Suggestion: 'inflatable slides' has query count: 59
Suggestion: 'infopass' has query count: 46
URL http://infopass.uscis.gov
Suggestion: 'infolanka' has query count: 40
URL http://www.infolanka.com
For query: 'inf' process time is: 0.02944756300000506


Suggestion: 'best buy' has query count: 685
URL http://www.bestbuy.com
Suggestion: 'bestcounter.biz' has query count: 257
Suggestion: 'bestbuy' has query count: 224
URL http://www.bestbuy.com
Suggestion: 'bestbuy.com' has query count: 158
URL http://www.bestbuy.com
Suggestion: 'best western' has query count: 99
URL http://www.bestwestern.com
For query: 'the best ' process time is: 0.03722066599999607


No suggestions for query: information retrieval
For query: 'information retrieval' process time is: 0.00011598900000819867


Sugg

## 2.4. [5] Question
What is the empirical threshold for minimal prefix length for suggest query? Write your answer in a cell bellow and justify it (just few sentences).

The average word length in English is 5. For queries consisting of frequent words the lenght of query 3-4 will be enought to fing more or less relevant results
But to increase correctness for not so frequent  queries we shouls take at least 6.

## 2.5. [+10] Add spellchecking to your suggest

Try to make your search results as close as possible. Compare top-5 results of each query with top-5 results for corrected.

In [19]:
from spellchecker import SpellChecker
import itertools

inp_queries = ["inormation retrieval", "shelrock hol", "carnagie mell", "babis r", "Barrbara "]
inp_queries_corrected = ["information retrieval", "sherlock hol", "carnegie mell", "babies r", "Barbara "]
spell = SpellChecker()

def complete_user_query_with_spellchecker(query, trie, top_k=5):
    words = query.split()
    suggestions = [list(spell.candidates(w)) for w in words]
    query = [" ".join(pr) for pr in  itertools.product(*suggestions)][0]
    print('\n')
    return complete_user_query(query , trie, top_k)

for q, qc in zip(inp_queries, inp_queries_corrected):
    assert  complete_user_query(qc, aol_trie, 5) == \
            complete_user_query_with_spellchecker(q, aol_trie, 5), "Assert {} and {} give different results".format(q, qc)

No suggestions for query: information retrieval


No suggestions for query: information retrieval
Suggestion: 'sherlock holmes' has query count: 4
URL http://www.sherlockian.net
Suggestion: 'sherlock holmes society' has query count: 2
URL http://www.realtime.net
Suggestion: 'sherlock holmes chronological order' has query count: 2
URL http://www.geocities.com
Suggestion: 'sherlock holmes address' has query count: 1
Suggestion: 'sherlock holmes audiotapes' has query count: 1


Suggestion: 'sherlock holmes' has query count: 4
URL http://www.sherlockian.net
Suggestion: 'sherlock holmes society' has query count: 2
URL http://www.realtime.net
Suggestion: 'sherlock holmes chronological order' has query count: 2
URL http://www.geocities.com
Suggestion: 'sherlock holmes address' has query count: 1
Suggestion: 'sherlock holmes audiotapes' has query count: 1
Suggestion: 'carnegie mellon' has query count: 6
URL http://www.cmu.edu
Suggestion: 'carnegie mellon university' has query count: 1
URL http

AssertionError: Assert babis r and babies r give different results